# Criação de um modelo de machine learning para prever as vendas

Temos 4 datasets, as transações para treino e para teste, as lojas, preços do óleo ao longo do tempo e os feriados.


Link do dataset
https://www.kaggle.com/competitions/store-sales-time-series-forecasting/overview

In [ ]:
#!pip install psycopg2

## 1 Coletando os dados

In [ ]:
# importando algumas bibliotecas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()

In [ ]:
import psycopg2

conn = psycopg2.connect(database='storesales', host='localhost', user='postgres', password='pokemonn123', port=5432 )
cur = conn.cursor()
cur.execute("""SELECT sl.date, sl.sales, sl.family, sl.onpromotion, sl.store_nbr, ol.dcoilwtico, st.city, st.state, st.cluster, st.type, tr.transactions
FROM sales sl
LEFT OUTER JOIN oil_prices ol
ON sl.date = ol.date
LEFT OUTER JOIN stores st
ON sl.store_nbr = st.store_nbr
LEFT OUTER JOIN transactions tr
ON sl.date = tr.date AND sl.store_nbr = tr.store_nbr
ORDER BY sl.date;""")
rows = cur.fetchall()
col_names = [desc[0] for desc in cur.description]

conn.commit()
conn.close()

#for row in rows:
#    print(row)

In [ ]:
df = pd.DataFrame(data=rows, columns=col_names)
df.shape

In [ ]:
df['date'] = pd.to_datetime(df['date'])
df.head()

In [ ]:
# Vamos utilizar o dataset de feriados no equador para saber se determinado dia foi feriado

conn = psycopg2.connect(database='storesales', host='localhost', user='postgres', password='pokemonn123', port=5432 )
cur = conn.cursor()
cur.execute("""SELECT  *
FROM holiday_events;
""")
rows = cur.fetchall()
col_names = [desc[0] for desc in cur.description]

conn.commit()
conn.close()

In [ ]:
df_hol = pd.DataFrame(data=rows, columns=col_names)
#df_hol.set_index('date',inplace=True)
df_hol['date'] = pd.to_datetime(df_hol['date'])

In [ ]:
df_hol.head(25)

## 2 Limpeza dos Dados

In [ ]:
df.isnull().sum()

In [ ]:
df[df.isnull()]['dcoilwtico']

In [ ]:
df['dcoilwtico'].plot()

In [ ]:
df['dcoilwtico'] = df['dcoilwtico'].bfill()

In [ ]:
df.isnull().sum()

In [ ]:
df['dcoilwtico'] = df['dcoilwtico'].astype('float32')

In [ ]:
df['dcoilwtico'].plot()

In [ ]:
median_transactions = df['transactions'].median()
df['transactions'] = df['transactions'].fillna(median_transactions)
df.isnull().sum()

In [ ]:
# Podemos ver que existem feriados nacionais, regionais (estaduais) e municipais
# Também pode haver feriados prolongados, ou o feriado pode ser transferido para outro dia
df_hol.head(5)

In [ ]:
# Agora vamos fazer um algoritmo para saber se determinado dia foi feriado e adicionar ao dataset
df['is_holiday'] = 0
for hol_index in df_hol[(df_hol['transferred']=='False') & (df_hol['hol_type'] != 'Work Day')].index:
    #if data_hol == data:
    holiday_date = df_hol.loc[hol_index, 'date']
    if holiday_date in df['date'].values:
        for i in  df[df['date']==holiday_date].index:
            if df_hol.loc[hol_index, 'locale'] == 'National':
                df.loc[i, 'is_holiday'] = 1
                
            elif (df_hol.loc[hol_index, 'locale'] == 'Regional') & (df_hol.loc[hol_index, 'locale_name'] == df.loc[i, 'state']):
                df.loc[i, 'is_holiday'] = 1

            elif (df_hol.loc[hol_index, 'locale']=='Local') & (df_hol.loc[hol_index, 'locale_name'] == df.loc[i, 'city']):
                df.loc[i,'is_holiday'] = 1
    
    else:
        pass

In [ ]:
df.head(25)

In [ ]:
df = df.sort_values(['store_nbr', 'family', 'date'])
df.head(25)

In [ ]:
df.groupby(['store_nbr','family'])['sales'].shift(1).rolling(window=7).mean()

In [ ]:
# Criação de lag_features:
df['lag_1'] = df.groupby(['store_nbr','family'])['sales'].shift(1)
df['lag_7'] = df.groupby(['store_nbr','family'])['sales'].shift(7)
df['sales_roll_mean_7'] = df.groupby(['store_nbr','family'])['sales'].shift(1).rolling(window=7).mean()
df['sales_roll_std_7'] = df.groupby(['store_nbr','family'])['sales'].shift(1).rolling(window=7).std()
df['promo_last_7'] = df.groupby(['store_nbr','family'])['onpromotion'].shift(1).rolling(window=7).sum()

df.head(25)

In [ ]:
df = df.fillna(0)
df.head(25)

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le = LabelEncoder()

categorical_cols = ['family', 'city', 'state', 'type', 'cluster']

# Encode each categorical column
for col in categorical_cols:
    
    df[col] = le.fit_transform(df[col])
    print(f"Encoded '{col}' column.")
    


In [ ]:
df.head()

In [ ]:
df['']

In [ ]:
# Normalizando os dados:

from sklearn.preprocessing import StandardScaler


numerical_features = ['onpromotion', 'dcoilwtico', 'lag_1', 'lag_7', 'sales_roll_mean_7', 'sales_roll_std_7' ,'promo_last_7', 'transactions']

scaler = StandardScaler()

df[numerical_features] = scaler.fit_transform(df[numerical_features])

df.head(25)

In [ ]:
# Salvando o DataFrame como CSV

df.to_csv('df_3.csv', index=True)

In [ ]:
# Importando o CSV:
df = pd.read_csv('df_3.csv', parse_dates=True, index_col='date')
df.head()

In [ ]:
sales = df['sales'].to_numpy().astype('float32')
#sales = np.log1p(sales)
raw_data = df.drop(columns=['sales', 'date']).astype('float32').to_numpy()

In [ ]:
print(sales.shape)
print(raw_data.shape)

## 3 Pré Processamento dos dados

In [ ]:
num_train_samples = int(0.75 * len(raw_data))
num_val_samples = int(0.125 * len(raw_data))
num_test_samples = len(raw_data) - num_train_samples - num_val_samples

print("num_train_samples:", num_train_samples)
print("num_val_samples:", num_val_samples)
print("num_test_samples:", num_test_samples)

## 4 Modelagem

In [ ]:
from sklearn.metrics import mean_absolute_error, r2_score, root_mean_squared_error
#from sklearn.model_selection import learning_curve
#from sklearn.model_selection import cross_val_score
#from sklearn.model_selection import TimeSeriesSplit

### 4.1 Dividindo o Dataset em features e label

In [ ]:
# Vamos dividir o nosso dataset em treino e teste
# Os dados de teste serão os últimos 15 dias
#train_dataset = df_pos.loc[:'2017-07-31']
#test_dataset = df_pos.loc['2017-08-01':]

In [ ]:
# Agora iremos dividir os dados em features e labels:

X_train = raw_data[: num_train_samples, :]
y_train = sales[ : num_train_samples]

X_val = raw_data [num_train_samples : num_train_samples + num_val_samples,  : ]
y_val = sales[num_train_samples : num_train_samples + num_val_samples]

X_test = raw_data[num_train_samples + num_val_samples: , :]
y_test = sales[num_train_samples + num_val_samples : ]

In [ ]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

### 4.2 Modelo Base

In [ ]:
# Será calculado um modelo base, sendo que este irá admitir que as vendas futuras serão iguais as vendas da última semana (7 dias atrás)
sales_df = pd.DataFrame({'sales':y_test})
sales_df['7_days'] = sales_df['sales'].shift(1782 * 7)
sales_df = sales_df.dropna(axis=0)

In [ ]:
print('MAE:', mean_absolute_error(sales_df['sales'], sales_df['7_days']))

### 4.3 Regrssão Linear

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lr = LinearRegression()

In [ ]:
lr.fit(X_train, y_train)

In [ ]:
predicts = lr.predict(X_val)
print(f"O valor do MAE é de: {mean_absolute_error(y_val, predicts)}")

### 4.4 Modelo SGDRegressor

In [ ]:
from sklearn.linear_model import SGDRegressor

In [ ]:
sgd = SGDRegressor()

In [ ]:
sgd.fit(X_train, y_train)

In [ ]:
predict = sgd.predict(X_val)
print('MAE:', mean_absolute_error(y_val, predict))

### 4.4 Modelo RandomForest

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rf = RandomForestRegressor()

In [ ]:
rf.fit(X_train, y_train)

In [ ]:
predict = rf.predict(X_val)

print('MAE:', mean_absolute_error(y_val, predict))

### 4.4 Criando Modelo XGBoost

In [ ]:
import xgboost as xgb

In [ ]:
# Treinando o modelo com XGBoost

clf = xgb.XGBRegressor(base_score=0.5, booster='gbtree',n_estimators=1000,objective='reg:squarederror', eval_metric='mae')

history = clf.fit(X_train, y_train ,eval_set=[(X_train, y_train), (X_val, y_val)])





In [ ]:
# Plotando a curva de validação
train_scores = history.evals_result()['validation_0']['mae']
val_scores = history.evals_result()['validation_1']['mae']
#history.evals_result()
epochs = len(val_scores)

plt.figure(figsize=(15,8))
plt.plot(range(1, epochs + 1), train_scores, 'r', label='Train MAE')
plt.plot(range(1, epochs + 1), val_scores, 'b', label='Validation MAE')
plt.title("Training and validation MAE")
plt.ylabel('MAE')
plt.xlabel("Epochs")
plt.legend();

In [ ]:
# O modelo está sofrendo de overfitting, agora será utilizado um gridsearch para encontrar os melhores parâmetros para o modelo:

In [ ]:
# Vamos usar early stopping para melhorar o overfitting:
# Treinando o modelo com XGBoost
from xgboost import XGBRegressor

clf = XGBRegressor(base_score=0.5, booster='gbtree',n_estimators=1000,objective='reg:squarederror',eval_metric='mae', early_stopping_rounds=50)

history = clf.fit(X_train, y_train ,eval_set=[(X_train, y_train), (X_val, y_val)], verbose=True)

print('Melhor score: ', history.best_score)
print('Melhor iteração: ',history.best_iteration)

# Plotando a curva de validação
train_scores = history.evals_result()['validation_0']['mae']
val_scores = history.evals_result()['validation_1']['mae']
#history.evals_result()
epochs = len(val_scores)

plt.figure(figsize=(15,8))
plt.plot(range(1, epochs + 1), train_scores, 'r', label='Train MAE')
plt.plot(range(1, epochs + 1), val_scores, 'b', label='Validation MAE')
plt.title("Training and validation MAE")
plt.ylabel('MAE')
plt.xlabel("Epochs")
plt.legend();


In [ ]:
gridsearch_params = [
(max_depth, eta, subsample, reg_lambda)
for max_depth in [2, 4, 6]
for eta in [0.01, 0.05, 0.1]
for subsample in [0.4, 0.6, 1]
for reg_lambda in [0, 0.5, 10]
]
mae_min = float("Inf")
best_params=None

for max_depth, eta, subsample, reg_lambda in gridsearch_params:

    # Atualiza os parâmetros:
    #params['max_depth'] = max_depth
    #params['eta'] = eta
    #params['subsample'] = subsample

    clf = XGBRegressor(base_score=0.5, booster='gbtree',n_estimators=1000,objective='reg:squarederror',eval_metric='mae', early_stopping_rounds=50, max_depth=max_depth, eta=eta, subsample=subsample, reg_lambda=reg_lambda)
    history = clf.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_val, y_val)], verbose=100)
    #train_scores = history.evals_result()['validation_0']['mae']
    #val_scores = history.evals_result()['validation_1']['mae']

    # Atualiza os melhores scores:
    mae = history.best_score
    if mae < mae_min:
        mae_min = mae
        best_params = (max_depth, eta, subsample, reg_lambda)

print(f'Melhores parâmetros (max_depth, eta, subsample e reg_lambda): {best_params[0]}, {best_params[1]}, {best_params[2]}, {best_params[3]}, e MAE: {mae_min} ')


In [ ]:
clf = xgb.XGBRegressor(base_score=0.5, booster='gbtree',n_estimators=1000,objective='reg:squarederror',eval_metric='mae', early_stopping_rounds=50, max_depth=6, eta=0.05, subsample=1,
                       gamma=0, reg_lambda=10)
history = clf.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_val, y_val)], verbose=100)

print('Melhor score: ', history.best_score)
print('Melhor iteração: ',history.best_iteration)

# Plotando a curva de validação
train_scores = history.evals_result()['validation_0']['mae']
val_scores = history.evals_result()['validation_1']['mae']
#history.evals_result()
epochs = len(val_scores)

plt.figure(figsize=(15,8))
plt.plot(range(1, epochs + 1), train_scores, 'r', label='Train MAE')
plt.plot(range(1, epochs + 1), val_scores, 'b', label='Validation MAE')
plt.title("Training and validation MAE")
plt.ylabel('MAE')
plt.xlabel("Epochs")
plt.legend();

In [ ]:
history.feature_importances_.shape

In [ ]:
pd.DataFrame(data=history.feature_importances_, index=df.drop(columns='sales').columns, columns=['importance'])

In [ ]:
# Plotando a importancia das features:
f1 = pd.DataFrame(data=history.feature_importances_, index=df.drop(columns='sales').columns, columns=['importance'])
f1 = f1.sort_values('importance', ascending=False)
f1[:20].plot(kind='barh', title='Feature Importance');

### 4.3 Criando Modelo com Rede Neural

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
inputs = keras.Input(shape= (82,), name='my_input')
features = layers.Dense(128, activation = 'relu', name='feature_1')(inputs)
features = layers.Dense(128, activation = 'relu', name='feature_2')(features)
outputs = layers.Dense(1, name='output')(features) 
model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='rmsprop', loss=['mean_squared_error'], metrics=['mae'])
model.fit(X_treino, y_treino)
model.evaluate()

In [ ]:
# Criação de rede neural com 2 camadas, 128 unidades por camada e batch size de 1024
# Será usado Blocking Time Series Split para Validação Cruzada
# Plotando a Learning Curve com Blocking Time Series Split

n_epochs= 100

inputs = keras.Input(shape= (83,), name='my_input')
features1 = layers.Dense(128, activation = 'relu', name='feature_1')(inputs)
features2 = layers.Dense(128, activation = 'relu', name='feature_2')(features1)
outputs = layers.Dense(1, name='output')(features2) 
model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer='rmsprop', loss='mean_squared_error', metrics=['mae'])
history = model.fit(X_train, y_train, epochs= n_epochs, validation_data=(X_val, y_val), batch_size= 1024, verbose=True)

mae_history_val = history.history['val_mae']
mae_history_train = history.history['mae']
loss_history_train = history.history['loss']
loss_history_val = history.history['val_loss']

    


# Plotando o MAE do treinamento e da validação
plt.figure(figsize=(15, 8))
plt.plot(range(1, len(mae_history_val) + 1), mae_history_val,'b' ,label='Validação')
plt.plot(range(1, len(mae_history_train) + 1), mae_history_train, 'r--' ,label='Treinamento')
plt.xlabel('Epochs')
plt.ylabel('MAE')
plt.legend()
plt.show()

# Plotando o Loss do treinamento e da validação
plt.figure(figsize=(15, 8))
plt.plot(range(1, len(loss_history_val) + 1), loss_history_val,'b' ,label='Validação')
plt.plot(range(1, len(loss_history_train) + 1), loss_history_train, 'r--' ,label='Treinamento')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


In [ ]:
# Será criado mais camadas e mais unidades por camada e também será adicionado dropout para regularização

callback_list = [keras.callbacks.ModelCheckpoint(filepath= 'sales_teste.keras', monitor='val_mae', save_best_only=True)]
n_epochs= 100
inputs = keras.Input(shape= (83,), name='my_input')
features1 = layers.Dense(512, activation = 'relu', name='feature_1')(inputs)
features1 = layers.Dropout(0.5)(features1)
features2 = layers.Dense(512, activation = 'relu', name='feature_2')(features1)
features2 = layers.Dropout(0.5)(features2)
features3 = layers.Dense(512, activation = 'relu', name='feature_3')(features2)
features3 = layers.Dropout(0.5)(features3)
features4 = layers.Dense(512, activation = 'relu', name='feature_4')(features3)
features4 = layers.Dropout(0.5)(features4)
features5 = layers.Dense(512, activation='relu', name='feature_5')(features4)
features5 = layers.Dropout(0.5)(features5)
outputs = layers.Dense(1, name='output')(features5) 
model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=0.0005), loss='mean_squared_error', metrics=['mae'])
history = model.fit(X_train, y_train, epochs= n_epochs, validation_data=(X_val, y_val), batch_size= 1024, callbacks=callback_list)



mae_history_val = history.history['val_mae']
mae_history_train = history.history['mae']
loss_history_train = history.history['loss']
loss_history_val = history.history['val_loss']

    


# Plotando o MAE do treinamento e da validação
plt.figure(figsize=(15, 8))
plt.plot(range(1, len(mae_history_val) + 1), mae_history_val,'b' ,label='Validação')
plt.plot(range(1, len(mae_history_train) + 1), mae_history_train, 'r--' ,label='Treinamento')
plt.xlabel('Epochs')
plt.ylabel('MAE')
plt.legend()
plt.show()

# Plotando o Loss do treinamento e da validação
plt.figure(figsize=(15, 8))
plt.plot(range(1, len(loss_history_val) + 1), loss_history_val,'b' ,label='Validação')
plt.plot(range(1, len(loss_history_train) + 1), loss_history_train, 'r--' ,label='Treinamento')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
model = keras.models.load_model("sales_teste.keras")
print(f"Test MAE: {model.evaluate(X_val, y_val)}")

### 5 Criando os modelos finais

Agora será criado os modelos finais para o algoritmo XGBoost e para a Rede neural, de acordo com os modelos criados anteiormente
Os modelos agora utilizarão todo o dataset de treino e validação para treinar.

In [ ]:
print(raw_data.shape)
print(sales.shape)

In [ ]:
# Desnormalizando o dataset
raw_data = (raw_data * std) + mean
raw_data

In [ ]:
 # Normalizando os dados novamente, mas agora levando em conta os dados de treino e validação
mean = raw_data[:num_train_samples + num_val_samples].mean(axis=0)
std = raw_data[:num_train_samples + num_val_samples].std(axis=0)
raw_data = (raw_data - mean) / std

In [ ]:
X_train = raw_data[: num_train_samples + num_val_samples, :]
y_train = sales[ : num_train_samples + num_val_samples]


X_test = raw_data[num_train_samples + num_val_samples: , :]
y_test = sales[num_train_samples + num_val_samples : ]

In [ ]:
from sklearn.ensemble import RandomForestRegressor
clf = RandomForestRegressor()
clf.fit(X_train, y_train)
predict = clf.predict(X_test)

print('MAE:', mean_absolute_error(y_test, predict))

In [ ]:
print('MAE:', mean_absolute_error(y_test, predict))

#### 5.1 Modelo final de XGBoost

In [ ]:
clf = xgb.XGBRegressor(base_score=0.5, booster='gbtree',n_estimators=1000,objective='reg:squarederror',eval_metric='mae', early_stopping_rounds=50, max_depth=6, eta=0.05, subsample=1,
                       gamma=0, reg_lambda=10)
history = clf.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)], verbose=100)

print('Melhor score: ', history.best_score)
print('Melhor iteração: ',history.best_iteration)


#### 5.2 Modelo final da Rede Neural

In [ ]:
# Será criado mais camadas e mais unidades por camada e também será adicionado dropout para regularização

callback_list = [keras.callbacks.ModelCheckpoint(filepath= 'sales_final.keras', monitor='val_mae', save_best_only=True)]
n_epochs= 100
inputs = keras.Input(shape= (83,), name='my_input')
features1 = layers.Dense(512, activation = 'relu', name='feature_1')(inputs)
features1 = layers.Dropout(0.5)(features1)
features2 = layers.Dense(512, activation = 'relu', name='feature_2')(features1)
features2 = layers.Dropout(0.5)(features2)
features3 = layers.Dense(512, activation = 'relu', name='feature_3')(features2)
features3 = layers.Dropout(0.5)(features3)
features4 = layers.Dense(512, activation = 'relu', name='feature_4')(features3)
features4 = layers.Dropout(0.5)(features4)
features5 = layers.Dense(512, activation='relu', name='feature_5')(features4)
features5 = layers.Dropout(0.5)(features5)
outputs = layers.Dense(1, name='output')(features5) 
model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=0.0005), loss='mean_squared_error', metrics=['mae'])
history = model.fit(X_train, y_train, epochs= n_epochs, validation_data=(X_test, y_test), batch_size= 1024, callbacks=callback_list)

In [ ]:
model = keras.models.load_model("sales_final.keras")
print(f"Test MAE: {model.evaluate(X_test, y_test)}")